# TRAIN YOUR OWN CUSTOM OBJECT DETECTOR

# 1. Create a new directory for training

## Directory Tree

- home/
    - models/
    - proto/
    - workspace/
        - Dockerfile
        - scripts/
            - image_downloader.py
            - xml_to_csv.py
            - create_tfrecords.py
        - training-directory/
            - annotations/
            - custom-models/
            - exported-models/
            - images/
                - test/
                - train/
            - pretrained-models/

***

- annotations/ - store the label map file(.pbtxt), the csv file (.csv) and corresponding TFRecord file (.record)
***
- images/ - copy of all the images (.jpg) in the dataset with corresponding xml files (.xml). image and xml files are created by a labelling software such as labelImg
***
- images/train/ - a copy of all the image and the corresponding xml files for training the model
***
- images/test/ - a copy of all the image and the corresponding xml files for testing the model
***
- custom-models/ - contains a sub directory for each of training job. Each subfolder will contain the training pipeline configuration file (.config) and other files generated during the training and evaluation of the model.
***
- pretrained-models/ - contains the downloaded pretrained models that will be used as a starting checkpoint to train the jobs. aka transfer learning.
***
- exported-models/ - store the exported version of the trained detector model

In [ ]:
# current working directory
import os
cwd = os.getcwd()

# training directory name
training_directory = "training-bear-SSD-MobileNet-V2-FPNLite-320x320"

In [ ]:
# list that contains the classes
labels = [
    "bear"
]

In [ ]:
# paths to be created inside the training directory
PATHS = {
    "training": "{}/{}".format(cwd, training_directory),
    "annotations": "{}/{}/annotations".format(cwd, training_directory),
    "custom-models": "{}/{}/custom-models".format(cwd, training_directory),
    "exported-models": "{}/{}/exported-models".format(cwd, training_directory),
    "images": "{}/{}/images".format(cwd, training_directory),
    "train-images": "{}/{}/images/train-images".format(cwd, training_directory),
    "test-images": "{}/{}/images/test-images".format(cwd, training_directory),
    "validation-images": "{}/{}/images/validation-images".format(cwd, training_directory),
    "pretrained-models": "{}/{}/pretrained-models".format(cwd, training_directory)
}

In [ ]:
# create the paths for each class to store train and test images
for label in labels:
    PATHS["{}-images".format(label)] = "{}/{}-images".format(PATHS["images"], label)

In [ ]:
PATHS

In [ ]:
# create the paths if they are not existing!

for path in PATHS.values():
    if not os.path.exists(path):
        !mkdir {path}

In [ ]:
# check if the directories inside PATHS have been created!
!ls {PATHS["training"]}

# 2. Prepare the dataset

## 2.1. Gather images
- image_downloader.py is a tool to install several images from google images
***
- -c   chromedriver path
***
- -k   keyword to search for images on google images
***
- -o   output path to write all the installed images
***
- -l   specify a level for the amount of images installed. Must be an integer, ranging from 1 to 5. The more the level is the more the amount of images to be installed.
***
- Or gather images from public datasets/databases, such as __[Open Images](https://storage.googleapis.com/openimages/web/index.html)__ and put those gathered images into the images folder for the corresponding object

In [ ]:
# image_downloader.py needs chromedriver, therefore find the executable path! 
!which chromedriver

In [ ]:
# location of the python script image_downloader.py
!ls scripts

In [ ]:
# exact path to the image_downloader.py script
!cd scripts && pwd

In [ ]:
# run this script to install images for each label
!python3 /home/workspace/scripts/image_downloader.py \
    -c /usr/local/bin/chromedriver \
    -k bear \
    -o {PATHS["bear-images"]} \
    -l 5

## 2.2 Annotate data
- Go and get __[labelImg](https://github.com/tzutalin/labelImg)__ and run labelImg.py, if you encounter a problem like not being able to write to a file then run labelImg.py with sudo.
***
- Open your images directory e.g. "home/workspace/training-dir/images" and start annotating the images inside of that folder.
***
- once you are done with annotating your images, in the images folder you should have an image file(.jpg) and an xml file(.xml) for each image that images directory.
***
- Do not create an xml file for the images that you do not wish to use for the training or testing. Next section will deal to delete those from the images directory.

## 2.3. Delete unused images

- If an image is not annotated, then it does not have a corresponding xml file. Delete those images which are not labelled.

In [ ]:
def get_images_to_remove(path):
    # get all the image, jpg files
    jpg_files = [file[:-4:1] for file in os.listdir(path) if file[-4::1] == ".jpg"]
    
    # get all the xml files, PASCAL VOC
    xml_files = [file[:-4:1] for file in os.listdir(path) if file[-4::1] == ".xml"]
    
    # compare if the jpg file has a corresponding xml file, otherwise append to the list
    jpg_files_to_remove = [jpg_file + ".jpg" for jpg_file in jpg_files if jpg_file not in xml_files]
    
    print("Number of total gathered images: {}".format(str(len(jpg_files))))
    print("Number of selected images: {}".format(str(len(xml_files))))
    print("Number of images to be removed: {}".format(str(len(jpg_files_to_remove))))
    
    return jpg_files_to_remove

In [ ]:
files_to_remove = {}

# loop through labels array to get the images that will be removed
# for each label class

for label in labels:
    print("Getting the images to be removed for {}".format(label))
    files_to_remove[label] = get_images_to_remove(PATHS["{}-images".format(label)])

In [ ]:
# loop through each label and remove the images to be removed

for label, files in files_to_remove.items():
    path = PATHS["{}-images".format(label)]
    print("removing the unused images from {} folder".format(path))
    print(len(files))
    for file in files:
        file_path = os.path.join(path, file)
        os.remove(file_path)

## 2.4. Split data into training and testing data

In [ ]:
import random

def get_training_testing_data(files_path, training_percentage = 85, validation_percentage = 10):
    training_percentage = float(training_percentage)
    validation_percentage = float(validation_percentage)
    
    num_of_training_data = (len(files_path) * training_percentage) // 100
    num_of_validation_data = (len(files_path) * validation_percentage) // 100
    
    training_data = random.sample(files_path,int(num_of_training_data))
    testing_and_validation_data = [e for e in files_path if e not in training_data]
    
    validation_data = random.sample(testing_and_validation_data,int(num_of_validation_data))
    
    testing_data = [e for e in testing_and_validation_data if e not in validation_data]
    
    return training_data, validation_data, testing_data

In [ ]:
import glob
import shutil

for label in labels:
    path = PATHS["{}-images".format(label)]
    # get all the xml files inside of that path
    xml_files = glob.glob(path + "/*.xml")
    file_paths = [file.replace(".xml", "") for file in xml_files]
    
    training_file_paths, validation_file_paths, testing_file_paths = get_training_testing_data(file_paths, 85, 10)
    
    # copy jpg and xml file for each image to training folder
    for file_path in training_file_paths:
        # copy jpg file
        shutil.copy(file_path + ".jpg", PATHS["train-images"])
        # copy xml file
        shutil.copy(file_path + ".xml", PATHS["train-images"])
    
    # copy jpg and xml file for each image to evaluation folder
    for file_path in validation_file_paths:
        # copy jpg file
        shutil.copy(file_path + ".jpg", PATHS["validation-images"])
        # copy xml file
        shutil.copy(file_path + ".xml", PATHS["validation-images"])
    
    # copy jpg and xml file for each image to test folder
    for file_path in testing_file_paths:
        # copy jpg file
        shutil.copy(file_path + ".jpg", PATHS["test-images"])
        # copy xml file
        shutil.copy(file_path + ".xml", PATHS["test-images"])

In [ ]:
!ls {PATHS["train-images"]}

In [ ]:
!ls {PATHS["validation-images"]}

In [ ]:
!ls {PATHS["test-images"]}

## 2.5. Create Label map file

- TensorFlow needs a label map file (.pbtxt)
- A labelmap file maps each of the used labels to an integer values.
- This label map is used both by the training and detection processes.
- Add your labels into the labels list

In [ ]:
# create a label map file under annotations
!touch {PATHS["annotations"]}/label_map.pbtxt

In [ ]:
!ls {PATHS["annotations"]}

In [ ]:
label_map_file = os.path.join(PATHS["annotations"], "label_map.pbtxt")

labelmaps = [None] * len(labels)

for idx, label in enumerate(labels):
    labelmaps[idx] = {"name": label, "id": idx + 1}

In [ ]:
labelmaps

In [ ]:
with open(label_map_file, "w") as f:
    for labelmap in labelmaps:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(labelmap['name']))
        f.write('\tid:{}\n'.format(labelmap['id']))
        f.write('}\n')

In [ ]:
!cat {PATHS["annotations"]}/label_map.pbtxt

## 2.6. Create TensorFlow Records

- Convert xml annotations to TFRecord format (.record).


In [ ]:
!ls

In [ ]:
!ls scripts

In [ ]:
!cd scripts && pwd

In [ ]:
xml_to_csv_path = "/home/workspace/scripts/xml_to_csv.py"
create_tfrecords_path = "/home/workspace/scripts/create_tfrecords.py"

In [ ]:
# create train.csv file in annotations folder
# run xml_to_csv.py to create a csv file for training data
!python3 {xml_to_csv_path} \
    -p {PATHS["train-images"]} \
    -o {PATHS["annotations"]}/train.csv

In [ ]:
!cat {PATHS["annotations"]}/train.csv

In [ ]:
# create evaluation.csv file in annotations folder
# run xml_to_csv.py to create a csv file for evaluation data
!python3 {xml_to_csv_path} \
    -p {PATHS["validation-images"]} \
    -o {PATHS["annotations"]}/validation.csv

In [ ]:
!cat {PATHS["annotations"]}/validation.csv

In [ ]:
# create test.csv file in annotations folder
# run xml_to_csv.py to create a csv file for test data
!python3 {xml_to_csv_path} \
    -p {PATHS["test-images"]} \
    -o {PATHS["annotations"]}/test.csv

In [ ]:
!cat {PATHS["annotations"]}/test.csv

In [ ]:
# create train.record under annotations folder
# run create_tfrecords.py to create a tfrecord file from the csv file
!python3 {create_tfrecords_path} \
    -l {PATHS["annotations"]}/label_map.pbtxt \
    -o {PATHS["annotations"]}/train.record \
    -i {PATHS["train-images"]} \
    -c {PATHS["annotations"]}/train.csv

In [ ]:
# create evaluation.record under annotations folder
# run create_tfrecords.py to create a tfrecord file from the csv file
!python3 {create_tfrecords_path} \
    -l {PATHS["annotations"]}/label_map.pbtxt \
    -o {PATHS["annotations"]}/validation.record \
    -i {PATHS["validation-images"]} \
    -c {PATHS["annotations"]}/validation.csv

In [ ]:
# create test.record under annotations folder
# run create_tfrecords.py to create a tfrecord file from the csv file
!python3 {create_tfrecords_path} \
    -l {PATHS["annotations"]}/label_map.pbtxt \
    -o {PATHS["annotations"]}/test.record \
    -i {PATHS["test-images"]} \
    -c {PATHS["annotations"]}/test.csv

# 3. Fine-tune the model

## 3.1. Selection of the pretrained object detection model
- TensorFlow Object Detection API provides several pretrained models under __[TensorFlow 2 Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md)__.
- Those models are trained with COCO dataset and can detect about 80 objects.
- Visit __[here](https://cocodataset.org/#home)__ to learn more about COCO dataset.
- All the models are trained using the same data but have different architecture, therefore, each model has its own corresponding speed, COCO mAP(Mean Average Precision).
- go to __[TensorFlow 2 Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md)__ and choose your model.

- This notebook is using "SSD MobileNet V2 FPNLite 320x320", change pretrained_model_url for your use case.

### Download the latest pretrained network for that model

In [ ]:
pretrained_model_url = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz"

In [ ]:
# INSTALL UNDER {PATHS["pretrained-models"]}
!wget -P {PATHS["pretrained-models"]} \
    {pretrained_model_url}

In [ ]:
# find the name of the installed tar
!ls {PATHS["pretrained-models"]}

In [ ]:
tar_folder_name = "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz"

In [ ]:
# UNZIP THE TAR
!tar -xvf {PATHS["pretrained-models"]}/{tar_folder_name} \
    -C {PATHS["pretrained-models"]}

In [ ]:
!ls {PATHS["pretrained-models"]}

In [ ]:
# REMOVE THE TAR
!rm -r {PATHS["pretrained-models"]}/{tar_folder_name}

In [ ]:
!ls {PATHS["pretrained-models"]}

In [ ]:
model_name = "/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8"

In [ ]:
# ADD THE RECENTLY INSTALLED PRETRAINED MODEL PATH TO THE PATHS DICTIONARY, c
PATHS["my-pretrained-model"] = PATHS["pretrained-models"] + model_name

In [ ]:
!ls {PATHS["my-pretrained-model"]}

In [ ]:
# ADD YOUR CUSTOM MODEL PATH TO THE PATHS DICTIONARY
PATHS["my-custom-model"] = PATHS["custom-models"] + model_name

In [ ]:
PATHS["my-custom-model"]

In [ ]:
# CREATE A DIRECTORY FOR YOUR CUSTOM MODEL
!mkdir {PATHS["my-custom-model"]}

In [ ]:
# copy pipeline config file from the pretrained model to the new folder
!cp {PATHS["my-pretrained-model"]}/pipeline.config \
    {PATHS["my-custom-model"]}

In [ ]:
!ls {PATHS["my-custom-model"]}

## 3.2. Configure the training pipeline

### Some of the important attributes
1. model.ssd.num_classes: number of classes
2. train_config.batch_size: Increase/Decrease this value depending on the available memory (Higher values require more memory and vice-versa) (e.g. 4)
3. train_config.fine_tune_checkpoint: Path to checkpoint of pre-trained model. (e.g. pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0")
4. train_config.fine_tune_checkpoint_type: Set this to "detection" if you are going to be training the model for detection
5. train_config.use_bfloat16: Set this to false if you are not training on a TPU
6. train_input_reader.label_map_path: Path to label map file. (e.g. annotations/label_map.pbtxt)
7. train_input_reader.tf_record_input_reader.input_path: Path to training TFRecord file (e.g. annotations/train.record)
8. eval_input_reader.label_map_path: Path to label map file (e.g. annotations/label_map.pbtxt)
9. eval_input_reader.tf_record_input_reader.input_path: Path to testing TFRecord (e.g. annotations/evalation.record)

In [ ]:
PATHS["my-pretrained-model"]

In [ ]:
!ls {PATHS["my-pretrained-model"]}/checkpoint

In [ ]:
PATHS["annotations"]

In [ ]:
!ls {PATHS["annotations"]}

In [ ]:
!cat {PATHS["my-custom-model"]}/pipeline.config

### Now open the pipeline.config file that is inside the custom model directory and edit it manually!

In [ ]:
# pipeline.config after configuration
!cat {PATHS["my-custom-model"]}/pipeline.config

In [ ]:
!ls

In [ ]:
!cd .. && ls

In [ ]:
!cd .. && cd models/research/object_detection && pwd

In [ ]:
# COPY model_main_tf2.py FILE TO THE TRAINING DIRECTORY
!cp /home/models/research/object_detection/model_main_tf2.py {PATHS["training"]}

In [ ]:
!ls {PATHS["training"]}

In [ ]:
!ls {PATHS["custom-models"]}

In [ ]:
!ls {PATHS["my-custom-model"]}

## 3.3. Start transfer learning
Run the training command from a terminal rather than running on this notebook. For that if you are on Linux go ahead and open up a new terminal and connect to that running docker container by following the steps below
<br>
<br>
1. get the id of the running container by inspecting the output of:
- docker ps
2. connect to that running docker container by executing(replace container_id with the running container's id):
- docker exec -i -t container_id bash
3. run the training command in a terminal, and run the validation command in another terminal

In [ ]:
# copy the command if you want to run the training command on terminal
pipeline_config_path = os.path.join(PATHS["my-custom-model"], "pipeline.config")
num_of_steps_per_checkpoint = 500

# training command
train_command = "cd {} && python3 model_main_tf2.py --model_dir={} --pipeline_config_path={} --checkpoint_every_n={}".format(PATHS["training"], PATHS["my-custom-model"], pipeline_config_path, int(num_of_steps_per_checkpoint))

# validation command
validation_command = "cd {} && python3 model_main_tf2.py --model_dir={} --pipeline_config_path={} --checkpoint_dir={} --sample_1_of_n_eval_examples=1".format(PATHS["training"], PATHS["my-custom-model"], pipeline_config_path, PATHS["my-custom-model"])

In [ ]:
print(train_command)

In [ ]:
print(validation_command)

In [ ]:
# once the training is done
# CUSTOM MODEL PATH SHOULD HAVE CHECKPOINTS, PIPELINE CONFIG, AND A TRAIN FOLDER
!ls {PATHS["MY_CUSTOM_MODEL"]}

### Monitoring the training job progress using TensorBoard

__[TensorBoard](https://www.tensorflow.org/tensorboard)__ allows you to coninuously monitor and visualise a number of different training/evaluation metrics, while your model is being trained.
<br>
<br>
1. Before starting a TensorBoard server, go ahead and connect to that running docker container from a different terminal by following the steps provided in the third section. Also make sure when you are running the docker container you bind at least 2 ports for the docker container, one for the jupyter notebook and another for the TensorBoard. You can also check available ports for that running container by inspecting:
- docker ps
<br>
<br>
2. Once you are connected to that running container start a new TensorBoard server by running:
- tensorboard --logdir=PATH_TO_MY_CUSTOM_MODEL --port=PORT --host 0.0.0.0
<br>
<br>
3. Command will start a new TensorBoard server, listening on the specified PORT of the docker container.
<br>
<br>
4. The command will output a url that you can use to go to the TensorBoard dashboard:


In [ ]:
port = 6006
tensorboard_command = "{} --logdir={} --port={} --host 0.0.0.0".format("tensorboard", PATHS["my-custom-model"], str(port))
print(tensorboard_command)

# 4. Object detection from checkpoint

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
import cv2 
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image

In [ ]:
!ls {PATHS["my-custom-model"]}

In [ ]:
!ls {PATHS["annotations"]}

In [ ]:
######### VARIABLES ###############
pipeline_config_path = "{}/pipeline.config".format(PATHS["my-custom-model"])
labelmap_path = "{}/label_map.pbtxt".format(PATHS["annotations"])
# get the latest checkpoint
checkpoint_file_path = "{}/ckpt-11".format(PATHS["my-custom-model"])

# build a detection model
configs = config_util.get_configs_from_pipeline_file(pipeline_config_path)
model_config = configs["model"]
detection_model = model_builder.build(model_config=model_config, is_training=False)

# restore checkpoint
checkpoint = tf.compat.v2.train.Checkpoint(model=detection_model)
checkpoint.restore(checkpoint_file_path).expect_partial()

@tf.function
def detect_fn(img_tensor):
    img_tensor, shapes = detection_model.preprocess(img_tensor)
    predictions = detection_model.predict(img_tensor, shapes)
    detections = detection_model.postprocess(predictions, shapes)
    return detections

category_index = label_map_util.create_category_index_from_labelmap(labelmap_path)

In [ ]:
def detect(img_path, result_file_path):
    img = cv2.imread(img_path)
    img_arr = np.array(img)
    
    # convert numpy array to tensor
    img_tensor = tf.convert_to_tensor(np.expand_dims(img_arr, 0), dtype=tf.float32)
    
    # get the objects in that tensor
    detections = detect_fn(img_tensor)
    
    num_of_detections = int(detections.pop('num_detections'))
    
    detections = {key: value[0, :num_of_detections].numpy() for key, value in detections.items()}
    
    detections['num_detections'] = num_of_detections
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    label_id_offset = 1
    img_arr_detections = img_arr.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
        img_arr_detections,
        detections["detection_boxes"],
        detections["detection_classes"] + label_id_offset,
        detections["detection_scores"],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=.5,
        min_score_thresh= 0.7,
        agnostic_mode=False)
    
    plt.imshow(cv2.cvtColor(img_arr_detections, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.savefig(result_file_path, bbox_inches='tight',pad_inches = 0)
    #plt.show()

In [ ]:
!ls {PATHS["test-images"]}

In [ ]:
# get all the jpg paths for testing from testing images
import glob
import random
img_paths = sorted(glob.glob(PATHS["test-images"] + "/*.jpg"))
print(img_paths)

In [ ]:
!ls {PATHS["bear-images"]}/bear-test

In [ ]:
# Create a folder to store the result jpg files
PATHS["detection-results-1-11ckpt"] = PATHS["images"] + "/detection-results-1-11ckpt"
!mkdir {PATHS["detection-results-1-11ckpt"]}

In [ ]:
for idx, img in enumerate(img_paths):
    detect(img, PATHS["detection-results-1-11ckpt"] + "/results-{}.jpg".format(idx))

In [ ]:
!ls {PATHS["training"]}/images

# If you are not satisfy with the model continue training

- to fine-tune the model use the latest checkpoint but create a new folder for the model.
- keep the confugations as the same except the train_input_reader

- change "train_config.fine_tune_checkpoint" to the latest checkpoint of the custom model. It was similar to "/home/workspace/training-bear/pretrained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0". Now change ckpt-0 to the latest such as ckpt-11.
- then start the training again using the training command

- Simply keep using the same configuration (except the train_input_reader) with the same model_dir of your previous model. That way, the API will create a graph and will check whether a checkpoint already exists in model_dir and fits the graph. If so - it will restore it and continue training it.

# References and Further studies

__[TensorFlow Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection)__

__[TensorFlow Model Garden](https://github.com/tensorflow/models)__

__[TensorFlow 2 Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md)__

__[TensorFlow 2 Object Detection API tutorial](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#create-label-map)__

__[labelImg annotation tool](https://github.com/tzutalin/labelImg)__

__[Object Detection from checkpoint](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/auto_examples/plot_object_detection_checkpoint.html)__

__[COCO dataset](https://cocodataset.org/#home)__

__[Object Detection From TF2 Saved Model](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/auto_examples/plot_object_detection_saved_model.html)__

__[Object Detection From TF2 Checkpoint](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/auto_examples/plot_object_detection_checkpoint.html)__

__[Detect Objects Using a Webcam](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/auto_examples/object_detection_camera.html)__

__[Training and Evaluating Custom Object Detectors](https://becominghuman.ai/tensorflow-object-detection-api-tutorial-training-and-evaluating-custom-object-detector-ed2594afcf73)__

__[TensorFlow Object Detection API: Best Practices to Training, Evaluation & Deployment](https://neptune.ai/blog/tensorflow-object-detection-api-best-practices-to-training-evaluation-deployment)__